In [ ]:
using Plots,Random,Measures,CSV,DataFrames,Statistics,Distributions
pyplot();

include("aux.jl"); include("poimcmc.jl");

In [ ]:
myrng = MersenneTwister(); 
prm,_=data();

In [ ]:
function myΓ(α::Float64,β::Float64,x::Float64)
    if x > 0
        val = exp(α*log(β))*exp((α-1)*log(x))*exp(-β*x)/gamma(α);
    else
        val = 0.0;
    end
    return val
end

function myΓ(α::Float64,β::Float64,x::Vector{Float64})
    n = length(x);
    val = Vector{Float64}(undef,n);
    for i=1:n
        val[i] = myΓ(α,β,x[i]);
    end
    
    return val
end;

In [ ]:
α = prm[:Γα]; β = prm[:Γβ]; μ=α/β; var = α/β^2;
prmrg,_ = mcmcrg();

Ax = convert(Vector,LinRange(0.001,250,10000));
Ay = myΓ(α,β,Ax);

println("Mean Γ-distribution for hyperparameters:")
println("(α,β)=($α,$β)")
println("(μ,var)=($μ,$var)")

### MCMC Approximate $\Gamma(\alpha,\beta)$

In [ ]:
nsmp = 50000; 

In [ ]:
"""
Density for normal distribution centered at x with std dev x/2
"""
function ρnrm(x::Float64,y::Float64)
    val = 1/√(2*pi)/(0.5*x);
    val *= exp(-0.5*( (x-y)/(0.5*x) )^2);
    
    return val
end;

function nrm(x::Float64,y::Float64,σ::Float64)
    val = 1/√(2*pi)/σ;
    val *= exp(-0.5*( (x-y)/(σ) )^2);
    
    return val
end;

In [ ]:
X = Vector{Float64}(undef,nsmp);
x0 = 750*rand(myrng);
for i=1:nsmp
    xprp = x0;
    coin = rand(myrng);
    if coin <= 0.5
        xprp += 0.5*x0*randn(myrng);
    elseif coin <=0.75
        xprp += 0.5*randn(myrng);
    else
        xprp += 375*randn(myrng);
    end
    
    mhratio = myΓ(α,β,xprp)/myΓ(α,β,x0);
    mhratio *= (0.5*ρnrm(xprp,x0)+0.25*nrm(xprp,x0,0.5)+0.25*nrm(xprp,x0,375.0))/(
                  0.5*ρnrm(x0,xprp)+0.25*nrm(x0,xprp,0.5)+0.25*nrm(x0,xprp,375.0));
    
    if rand(myrng)<= mhratio
        x0 = xprp;
    end
    
    X[i]=x0;
end

In [ ]:
plot(Ax,Ay,linewidth=3,title="Empirical Γ-distr",labels="theory",size=(300,250))
histogram!(X,normalize=:pdf,xlims=(0,5),labels="MCMC",alpha=0.5,bins=[0,0.05,0.1,0.5,5])

### Mean shedding from n-individuals by bootstrap

In [ ]:
n = 80;
L = 7.0; t₀=0.0; T=7.0; Aₓ=3.5; ξ=log(2);

In [ ]:
#### Bootstrap Gamma
SMP = Matrix{Float64}(undef,n,nsmp);
for i=1:nsmp
    for j=1:n
        A = X[rand(myrng,1:nsmp)];
        SMP[j,i] = shedλ(A,L,t₀,T,Aₓ,ξ);
    end
end
SMP = sum(SMP,dims=1) |> (x->reshape(x,nsmp));

In [ ]:
#### For direct Γ
ΓSMP = Matrix{Float64}(undef,n,nsmp);
for i=1:nsmp
    for j=1:n
        A = rand(myrng,Gamma(α,1/β));
        ΓSMP[j,i] = shedλ(A,L,t₀,T,Aₓ,ξ);
    end
end
ΓSMP = sum(ΓSMP,dims=1) |> (x->reshape(x,nsmp));

In [ ]:
histogram(SMP[1:1:end],normalize=:pdf,title="Shedding Poisson Mean $n-indiv with nsmp=$nsmp",label="MC SMP",bins=100)
histogram!(ΓSMP[1:1:end],normalize=:pdf,label="Γ SMP",alpha=0.5,bins=100,margin=1mm)

In [ ]:
savefig("scratch.pdf")

In [ ]:
println("MC Mean Poisson Mean:");
println(sum(SMP)/length(SMP));
println("");
println("Γ Mean Poisson Mean:");
println(sum(ΓSMP)/length(ΓSMP))
println("")
println("Matt said Γ mean of means should be this when ξ=0:")
println(0.5*7*α/β*80)

#### Compare with Matt's Synthetic

In [ ]:
val = 150;

Matt's probabilities:<br>
copies | n <br>
237	80 <br>
77	80 <br>
144	80 <br>

In [ ]:
"""
Find cdf of poisson evaluated at q
"""
function cdfpoi(μ::Float64,q::Int64)
    val = exp(-μ);
    if q>0
        for i=1:q
            val += exp(-μ)*prod(fill(μ,i)./(1:i));
        end
    end
    
    return val
end;

In [ ]:
q5 = (round(quantile(SMP,0.5)))

In [ ]:
println("Empirical probability of falling below $val for 50% quantile Poisson mean:")
println(cdfpoi(q5,val))